In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from netam import noof, shmoof
from epam.torch_common import pick_device

In [2]:
all_df = pd.read_csv("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv")

(all_df["parent"].str.len() > 410).sum() / len(all_df)

0.00403216242498992

In [3]:
train_df, val_df = shmoof.load_shmoof_dataframes("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv", sample_count=5000)

In [4]:
kmer_length = 5
max_length = 410

train_dataset = shmoof.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = shmoof.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

we have 3704 training examples and 1296 validation examples


In [5]:
model = noof.NoofModel(train_dataset, embedding_dim=1, nhead=1, dim_feedforward=512, layer_count=3, dropout=0.1)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)
model.to(device)

burrito = shmoof.NoofBurrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=10)

Using Metal Performance Shaders
starting training...


/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/10]	 Loss: 0.063052209	 Val Loss: 0.072215926
Epoch [2/10]	 Loss: 0.06304868	 Val Loss: 0.072200712
Epoch [3/10]	 Loss: 0.063044176	 Val Loss: 0.072207694
Epoch [4/10]	 Loss: 0.063044291	 Val Loss: 0.072196442
Epoch [5/10]	 Loss: 0.063041077	 Val Loss: 0.072194734
Epoch [6/10]	 Loss: 0.063040823	 Val Loss: 0.072195488


In [ ]:
torch.save(model, "_ignore/noof_model.pt")